<a href="https://colab.research.google.com/github/Kanth-Somala/AI-Haiku-Poem-Generator/blob/main/AI_Genrated_Haiku_Poems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta 0:00:00


In [ ]:
# Load haiku file
file_path = '/content/lines.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()

# Split by '$' and convert each haiku into 3-line format
raw_haikus = [h.strip() for h in raw_text.split('$') if h.strip()]

formatted_haikus = []
for h in raw_haikus:
    parts = [line.strip() for line in h.split('/') if line.strip()]
    if len(parts) == 3:
        formatted_haikus.append('\n'.join(parts))  # maintain 3-line format


In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict({"text": formatted_haikus})


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=50)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function)


Map:   0%|          | 0/11266 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir="./haiku-gpt2",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    logging_steps=10,
    save_strategy="epoch",
    fp16=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


In [ ]:
trainer.train()

Step,Training Loss
10,3.008400
20,1.747100
30,1.626100
40,1.623100
50,1.544700
60,1.557500
70,1.546300
80,1.460500
90,1.495900
100,1.608800


TrainOutput(global_step=14085, training_loss=1.1579814044896872, metrics={'train_runtime': 1612.8107, 'train_samples_per_second': 34.927, 'train_steps_per_second': 8.733, 'total_flos': 1437361344000000.0, 'train_loss': 1.1579814044896872, 'epoch': 5.0})

In [ ]:
model.save_pretrained("./haiku-gpt2-finetuned")
tokenizer.save_pretrained("./haiku-gpt2-finetuned")

('./haiku-gpt2-finetuned/tokenizer_config.json',
 './haiku-gpt2-finetuned/special_tokens_map.json',
 './haiku-gpt2-finetuned/vocab.json',
 './haiku-gpt2-finetuned/merges.txt',
 './haiku-gpt2-finetuned/added_tokens.json',
 './haiku-gpt2-finetuned/tokenizer.json')

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./haiku-gpt2-finetuned", tokenizer="./haiku-gpt2-finetuned")

prompt = "nature is awsome"
outputs = generator(prompt, max_new_tokens=40, num_return_sequences=3, do_sample=True, top_k=50)

for i, output in enumerate(outputs):
    print(f"\nHaiku {i+1}:\n{output['generated_text']}")


Device set to use cuda:0



Haiku 1:
nature is awsome
toasty tea leaves shimmer
warmth escapes my grasp

Haiku 2:
nature is awsome
nature has wisdom beyond
nature is beautiful

Haiku 3:
nature is awsome
i hope you enjoy it
goodnight sir


In [ ]:
!pip install syllables --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 23.8 MB/s eta 0:00:00


In [ ]:
import syllables
from transformers import pipeline

prompt = input("Enter a haiku theme: ")

generator = pipeline(
    "text-generation",
    model="./haiku-gpt2-finetuned",
    tokenizer="./haiku-gpt2-finetuned"
)

outputs = generator(
    prompt,
    max_new_tokens=40,
    num_return_sequences=100,
    do_sample=True,
    top_k=50
)

def count_syllables(line):
    return sum(syllables.estimate(word) for word in line.split())

def is_valid_575(lines):
    return (
        len(lines) == 3 and
        count_syllables(lines[0]) == 5 and
        count_syllables(lines[1]) == 7 and
        count_syllables(lines[2]) == 5
    )

def extract_haiku(text):
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    if is_valid_575(lines):
        return lines
    words = text.strip().split()
    if len(words) < 17:
        return None
    candidate_lines = [
        " ".join(words[:5]),
        " ".join(words[5:12]),
        " ".join(words[12:17])
    ]
    if is_valid_575(candidate_lines):
        return candidate_lines
    return None

valid_haikus = []
for output in outputs:
    text = output["generated_text"]
    haiku = extract_haiku(text)
    if haiku:
        valid_haikus.append(haiku)
        if len(valid_haikus) >= 3:
            break

if valid_haikus:
    print("\n Haikus Generated:")
    for idx, haiku in enumerate(valid_haikus, 1):
        print(f"\n🌿 Haiku {idx}:")
        for line in haiku:
            print(f"{line}")
else:
    print("\n❌ No valid 5-7-5 haikus were found. Try another theme or generate more.")


Enter a haiku theme: winter breeze


Device set to use cuda:0



 Haikus Generated:

🌿 Haiku 1:
winter breeze blows
the leaves sing out their songs
winter is coming

🌿 Haiku 2:
winter breeze blows
the leaves fall from the trees
to form a new tree

🌿 Haiku 3:
winter breeze blows
the leaves fall from the trees
the leaves fall still


In [ ]:
syllables.estimate('becarefulfolks')

5

In [2]:
!pip install nbstripout
!nbstripout AI Genrated Haiku Poems.ipynb


Could not strip 'Poems.ipynb': file not found
